In [1]:
import urllib3
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from io import StringIO
import json
from headers import *
import controllers

import importlib
import models
importlib.reload(controllers)
importlib.reload(models)
controller = controllers.FuturesDataController()
model = models.FuturesDataModel()



In [ ]:
date = "20240830"
url = "https://www.twse.com.tw/rwd/zh/TAIEX/MI_5MINS_HIST"
params = {
            'response':'json',
            'date':'20240830'
        }
b = requests.post(url,data=params,headers=TWSE_DOWN)
b.text

In [ ]:
controller.update_twse_data("2024/08/30")

In [19]:
controller.update_taifex_data("2024/09/11")

爬取日期: 2024/09/11
爬取成功
轉換成功


/Users/hayden/Documents/pyDB/controllers.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter[col] = pd.to_numeric(df_filter[col].astype(str).str.replace(',', ''), errors='coerce')
/Users/hayden/Documents/pyDB/controllers.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter[col] = pd.to_numeric(df_filter[col].astype(str).str.replace(',', ''), errors='coerce')
/Users/hayden/Documents/pyDB/controllers.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

成功連接到 MongoDB!
成功插入 66e15139370ca68770dcc34a 條文檔
MongoDB 連接已關閉


'成功更新 9 條記錄到 MongoDB'

In [ ]:
df2 = controller.crawler_twse_data("2024/08/30")
df2

In [ ]:
df3 = controller.crawler_twse_total_return("2024/08/30")
df3

In [ ]:
merged_df = pd.merge(df2,df3,on='date',how='outer')
merged_df

In [ ]:
merged_df

In [ ]:
df_dates = set(merged_df['date'])
print(df_dates)

In [ ]:
client = model.connect_mongo()

In [ ]:
a = merged_df.to_dict('records')
print(a)

In [ ]:
a

In [11]:
db = client["findata"]
collection = db["twse_index"]

result = collection.insert_many(a)

In [ ]:
controller.crawler_taifex_data("2024/09/09")

In [ ]:
url = "https://www.taifex.com.tw/cht/3/futContractsDate"
        
params = {
       'queryType': '2',
       'goDay': '',
       'doQuery': '1',
       'dateaddcnt': '',
       'queryDate': "2024/09/08"
   }
response = requests.post(url,headers=TAIFEX_OI,data=params)
response.text

In [ ]:
soup = BeautifulSoup(response.content,'html.parser')
table = soup.table
df = pd.read_html(StringIO(str(table)))
df = df[0]
df

In [ ]:
df.drop(columns = df.columns[0],axis=1,inplace=True)
df

In [ ]:
len(df)

In [5]:
date = "2024/09/09"
data = {
            "date":datetime.strptime(date,"%Y/%m/%d"),
            "期貨契約":[]
        }
for _, row in df.iterrows():
    contract = {
        "商品名稱": row['商品名稱'],
        "身份別": row['身份別'],
        "多方交易口數": int(row['多方交易口數']),
        "多方交易金額": int(row['多方交易金額']),
        "空方交易口數": int(row['空方交易口數']),
        "空方交易金額": int(row['空方交易金額']),
        "交易淨口數": int(row['交易淨口數']),
        "交易淨金額": int(row['交易淨金額']),
        "多方未平倉口數": int(row['多方未平倉口數']),
        "多方未平倉金額": int(row['多方未平倉金額']),
        "空方未平倉口數": int(row['空方未平倉口數']),
        "空方未平倉金額": int(row['空方未平倉金額']),
        "未平倉淨口數": int(row['未平倉淨口數']),
        "未平倉淨金額": int(row['未平倉淨金額'])
    }
    data["期貨契約"].append(contract)
    

In [ ]:
print(data)

In [ ]:
client = model.connect_mongo()

In [11]:
db = client["findata"]
collection = db["txf_oi"]


In [12]:
existing_data = collection.find_one({'date':data['date']})

In [ ]:
print(existing_data)
print(data)

In [17]:
result = collection.insert_one(data)

In [ ]:
df = controller.crawler_twse_total_return("20240909")
df

In [ ]:
df2 = controller.crawler_twse_data("20240909")
df2

In [2]:
params ={
    'response':'json',
    'date':"20240810"
}
rsp = requests.get("https://www.twse.com.tw/rwd/zh/TAIEX/MFI94U",data=params,headers=TWSE_DOWN)

In [ ]:
d2 = json.loads(rsp.text)
d2

In [ ]:
df2 = pd.DataFrame(d2['data'],columns=['date','tri_close'])
df2

In [ ]:
df2.iloc[:,1:] = df2.iloc[:,1:].apply(lambda x: x.str.replace(',','').astype(float), axis=1)
df2

In [ ]:
df2[['year','month','day']] = df2['date'].str.split('/',expand=True)
df2['year'] = df2['year'].astype(int)+1911
df2['date'] = df2['year'].astype(str)+'-'+df2['month'].str.zfill(2) + "-" + df2['day'].str.zfill(2)
df2['date'] = pd.to_datetime(df2['date'])
df2

In [ ]:
#tdf = controller.crawler_twse_data("20240801")
xdf = controller.crawler_twse_data("20240801")

In [ ]:
xdf

In [ ]:
date = "20240905"
url = "https://www.twse.com.tw/indicesReport/MI_5MINS_HIST"
params = {
    'response':'json',
    'date':f'{date}'
}

response = requests.get(url,data=params,headers=TWSE_DOWN)


In [ ]:
response.text

In [ ]:
data = json.loads(response.text)
data

In [ ]:
df = pd.DataFrame(data['data'],columns=['date','open','high','low','close'])
df

In [ ]:
df.iloc[:, 1:] = df.iloc[:, 1:].apply(lambda x: x.str.replace(',', '').astype(float), axis=1)

In [ ]:
df

In [ ]:
df[['year', 'month', 'day']] = df['date'].str.split('/', expand=True)
df['year'] = df['year'].astype(int) + 1911
df['date'] = df['year'].astype(str) + '-' + df['month'].str.zfill(2) + '-' + df['day'].str.zfill(2)
df

In [ ]:
df[['year','Month', 'Day']] = df['Date'].str.split("/",expand=True,)


In [ ]:
df

In [ ]:
df['Year'] = df['Year'].astype(int) + 1911


In [ ]:
df

In [ ]:
df['Date']  = (pd.to_datetime(df['Year'].astype(str) + '/' +
df['Month'].astype(str) + '/' +
df['Day'].astype(str)))

In [ ]:
df

In [ ]:
df = df.iloc[:, :-3]







In [ ]:
df